In [1]:
#Import your libraries

import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
import pylab as plt
import seaborn as sns
import difflib

In [2]:
obras_completo_original = pd.read_csv('/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/sparql_2023-11-30_10-13-22Z.csv')

obras_completo = obras_completo_original.copy()

# Estudio de la información de iconografía en el Museo Del Prado.

# 1. Exploración inicial.

In [3]:
obras_completo.head()

,uriObra,tituloObra,autores,fecha,anio,Escuelas,Objetos,Faunas,Floras,LugaresGeo,Personajes,Imagenes
0,http://museodelprado.es/items/E22_Man-Made_Obj...,Floreros,"Nuzzi, Mario || Nuzzi, Mario (Mario dei Fiori)",1640 - 1642,1640,Italiana,Enseres domésticos || Florero || Instrumentos ...,NaN,Anémona (Anemone spp.) || Gladiolo (Gladiolus ...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...
1,http://museodelprado.es/items/E22_Man-Made_Obj...,La Anunciación,Maestro de la Sisla || Master of La Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,https://content3.cdnprado.net/imagenes/Documen...
2,http://museodelprado.es/items/E22_Man-Made_Obj...,Copa de la Vendimia,"Fontana, Annibale || Taller de los Sarachi || ...",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...
3,http://museodelprado.es/items/E22_Man-Made_Obj...,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) || Anémo...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...
4,http://museodelprado.es/items/E22_Man-Made_Obj...,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios || Insignia...,NaN,Azucena (Lilium candidum),NaN,Virgen María || Ángel / ángeles,https://content3.cdnprado.net/imagenes/Documen...


In [4]:
dim_original = obras_completo.shape # De esta forma guardo las dimensiones del data frame original.

obras_completo.shape

(655, 12)

In [5]:
obras_completo.columns # Esto nos proporciona una enumeración de los nombres correspondientes a cada una de las columnas presentes en los datos.

Index(['uriObra', 'tituloObra', 'autores', 'fecha', 'anio', 'Escuelas',
       'Objetos', 'Faunas', 'Floras', 'LugaresGeo', 'Personajes', 'Imagenes'],
      dtype='object')

In [6]:
obras_completo.info() # De este modo conzco el tipo de dato y los valores nulos que tiene cada una de las columnas.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uriObra     655 non-null    object
 1   tituloObra  655 non-null    object
 2   autores     655 non-null    object
 3   fecha       655 non-null    object
 4   anio        655 non-null    int64 
 5   Escuelas    649 non-null    object
 6   Objetos     319 non-null    object
 7   Faunas      188 non-null    object
 8   Floras      655 non-null    object
 9   LugaresGeo  48 non-null     object
 10  Personajes  309 non-null    object
 11  Imagenes    655 non-null    object
dtypes: int64(1), object(11)
memory usage: 61.5+ KB


In [7]:
# Renombrar las columnas
obras_completo = obras_completo.rename(columns={
    'uriObra': 'Obra_uri',
    'tituloObra': 'Título',
    'autores': 'Autor',
    'fecha': 'Fecha',
    'anio': 'Año',
    'Floras': 'Flora',
    'Escuelas': 'Escuela',
    'Faunas': 'Fauna',
    'LugaresGeo': 'Lugar',
    'Imagenes': 'Imágenes'
})

In [8]:
columnas_a_procesar = ['Autor', 'Flora', 'Imágenes', 'Escuela', 'Objetos', 'Fauna', 'Lugar', 'Personajes']

# Reemplazar "||" solo si está entre dos palabras o fragmentos de palabras
obras_completo[columnas_a_procesar] = obras_completo[columnas_a_procesar].apply(lambda x: x.str.replace(r'(?<= )\|\|(?= )', ';'))

# Análisis de "Fecha" y "Año"

In [9]:
#obras_completo['Fecha'].unique()

In [10]:
fecha_año = obras_completo[['Fecha', 'Año']]

In [11]:
fecha_año.head()

,Fecha,Año
0,1640 - 1642,1640
1,Hacia 1500,1495
2,1570 - 1600,1570
3,Hacia 1676,1671
4,Hacia 1781,1776


In [12]:
"""
Lo que voy a hacer es, en primer lugar, dejar la columna de fecha porque solo tiene un valor numérico, lo cual
nos permite hacer operaciones con ella. También dejaré la de Fecha simplemnete para poder consultar esa información
y luego haré una nueva que contenga un rango, para luego poder crear la línea del tiempo. La denomnaré rango_fecha
""" 

'\nLo que voy a hacer es, en primer lugar, dejar la columna de fecha porque solo tiene un valor numérico, lo cual\nnos permite hacer operaciones con ella. También dejaré la de Fecha simplemnete para poder consultar esa información\ny luego haré una nueva que contenga un rango, para luego poder crear la línea del tiempo. La denomnaré rango_fecha\n'

In [13]:
"""
Para ello dejaré tal cual los valores que ya sean un rango en fecha y en caso de que no lo sean los analizaré
"""

'\nPara ello dejaré tal cual los valores que ya sean un rango en fecha y en caso de que no lo sean los analizaré\n'

In [14]:
fecha_año[fecha_año['Fecha'].str.contains('Siglo I a.C.')]

,Fecha,Año
207,Siglo I a.C.,-10


In [15]:
def transformar_fechas(fecha):
    if 'Primer cuarto del siglo XVII' in fecha:
        return '1601 - 1625'
    elif 'Siglo XVII' in fecha:
        return '1601 - 1700'
    elif 'Finales del siglo XVII - Principio del siglo XVIII' in fecha:
        return '1685 - 1715'
    elif 'Finales del siglo III a.C. - Primera mitad del siglo II a.C.' in fecha:
        return '-30 - -25'
    elif 'Siglo XVI' in fecha:
        return '1501 - 1600'
    elif 'Segunda mitad del siglo XVI' in fecha:
        return '1551 - 1600'
    elif 'Principio del siglo XX' in fecha:
        return '1901 - 1915'
    elif 'Último tercio del siglo XVIII' in fecha:
        return '1769 - 1800'
    elif 'Finales del siglo XVI - Primer tercio del siglo XVII' in fecha:
        return '1585 - 1630'
    elif 'Primera mitad del siglo XVII' in fecha:
        return '1601 - 1650'
    elif 'Segunda mitad del siglo XVII' in fecha:
        return '1651 - 1700'
    elif 'Finales del siglo XVII' in fecha:
        return '1685 - 1700'
    elif 'Siglo XVIII' in fecha:
        return '1701 - 1800'
    elif 'Finales del siglo XVIII' in fecha:
        return '1785 - 1800'
    elif 'Siglos XVI - XVII' in fecha:
        return '1501 - 1700'
    elif 'Segunda mitad del siglo XVII - Primer cuarto del siglo XVIII' in fecha:
        return '1651 - 1725'
    elif 'Siglo I a.C.' in fecha:
        return '-100'
    elif 'Finales del siglo XVI' in fecha:
        return '1685 - 1700'
    elif 'Primer tercio del siglo XVII' in fecha:
        return '1601 - 1630'
    elif 'Siglo III a.C.' in fecha:
        return '-300'
    elif 'Primer tercio del siglo XVI' in fecha:
        return '1501 - 1530'
    elif 'Segundo tercio del siglo XVII' in fecha:
        return '1634 - 1870'
    elif 'Siglo XII' in fecha:
        return '1101 - 1199'
    elif 'Último cuarto del siglo XVIII - Principio del siglo XIX' in fecha:
        return '1776 - 1815'
    elif 'Último cuarto del siglo XVII' in fecha:
        return '1676 - 1700'
    elif 'Siglos IV a.C. - III a.C.' in fecha:
        return '-400 - -300'
    elif 'Mediados del siglo XVII' in fecha:
        return '1640 - 1660'
    elif 'Siglo XIX' in fecha:
        return '1801 - 1900'
    
    else:
        return fecha

fecha_año['rango_fecha'] = fecha_año['Fecha'].apply(transformar_fechas)

In [16]:
#si pone Hacia...poner el valor que viene en Año:
fecha_año['Fecha_rango'] = fecha_año.apply(lambda row: row['Año'] if 'Hacia' in row['Fecha'] else row['Fecha'], axis=1)

In [17]:
fecha_año.head()

,Fecha,Año,rango_fecha,Fecha_rango
0,1640 - 1642,1640,1640 - 1642,1640 - 1642
1,Hacia 1500,1495,Hacia 1500,1495
2,1570 - 1600,1570,1570 - 1600,1570 - 1600
3,Hacia 1676,1671,Hacia 1676,1671
4,Hacia 1781,1776,Hacia 1781,1776


In [18]:
#fecha_año['rango_fecha'].unique()

In [19]:
obras_completo['Fecha_rango'] = fecha_año['Fecha_rango']

In [20]:
obras_completo.head()

,Obra_uri,Título,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,Imágenes,Fecha_rango
0,http://museodelprado.es/items/E22_Man-Made_Obj...,Floreros,"Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,http://museodelprado.es/items/E22_Man-Made_Obj...,La Anunciación,Maestro de la Sisla ; Master of La Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,https://content3.cdnprado.net/imagenes/Documen...,1495
2,http://museodelprado.es/items/E22_Man-Made_Obj...,Copa de la Vendimia,"Fontana, Annibale ; Taller de los Sarachi ; Wo...",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,http://museodelprado.es/items/E22_Man-Made_Obj...,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) ; Anémon...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1671
4,http://museodelprado.es/items/E22_Man-Made_Obj...,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,NaN,Azucena (Lilium candidum),NaN,Virgen María ; Ángel / ángeles,https://content3.cdnprado.net/imagenes/Documen...,1776


In [21]:
def transformar_años(año):
    if año == -10:
        return -100
    elif año == -20:
        return -200
    elif año == -30:
        return -300
    elif año == -40:
        return -400
    else:
        return año

obras_completo['Año'] = obras_completo['Año'].apply(transformar_años)

In [22]:
obras_completo['Año'].unique()

array([1640, 1495, 1570, 1671, 1776, 1601, 1924, 1856, 1727, 1615, 1663,
       1505, 1642, 1908, 1832, 1660, 1689, 1550, 1874, 1650, 1685,  -21,
       1787, 1635, 1859, 1501, 1515, 1545, 1906, 1490, 1885, 1585, 1659,
       1867, 1551, 1799, 1805, 1826, 1425, 1829, 1595, 1783, 1654, 1780,
       1847, 1670, 1901, 1919, 1628, 1779, 1620, 1751, 1872, 1846, 1634,
       1769, 1633, 1857, 1643, 1630, 1655, 1913, 1804, 1631, 1600, 1868,
       1870, 1915, 1603, 1900, 1811, 1541, 1910, 1658, 1795, 1692, 1627,
       1651, 1803, 1724, 1875, 1423, 1624, 1871, 1869, 1888, 1675, 1695,
       1680, 1690, 1719, 1567,   50, 1701, 1770, 1890, 1790, 1876, 1694,
       1610, 1785, 1898, 1914, 1858, 1882, 1881, 1844, 1616, 1806, 1668,
       1612, 1784, -100, 1866, 1877, 1879, 1862, 1678, 1676, 1617, 1887,
       1813, 1638, 1531, 1491, 1664, 1745, 1781, -300, 1892, 1665, 1606,
       1775, 1465, 1644, 1604, 1860, 1509, 1895, 1789, 1764, 1760, 1884,
       1837, 1774, 1480, 1801, 1969, 1645, 1842, 18

# Análisis "Autor"

In [23]:
obras_completo['Título'] = obras_completo['Título'].apply(lambda x: f'"{x}"')

In [24]:
obras_completo.head()

,Obra_uri,Título,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,Imágenes,Fecha_rango
0,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Floreros""","Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,http://museodelprado.es/items/E22_Man-Made_Obj...,"""La Anunciación""",Maestro de la Sisla ; Master of La Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,https://content3.cdnprado.net/imagenes/Documen...,1495
2,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Copa de la Vendimia""","Fontana, Annibale ; Taller de los Sarachi ; Wo...",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Florero""","Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) ; Anémon...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1671
4,http://museodelprado.es/items/E22_Man-Made_Obj...,"""La Inmaculada Concepción""","Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,NaN,Azucena (Lilium candidum),NaN,Virgen María ; Ángel / ángeles,https://content3.cdnprado.net/imagenes/Documen...,1776


In [25]:
#obras_completo['Autor'].unique()

In [26]:
nombres_unicos = obras_completo['Autor'].unique()
nombres_unicos = list(nombres_unicos)
nombres_unicos

['Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)',
 'Maestro de la Sisla ; Master of La Sisla',
 'Fontana, Annibale ; Taller de los Sarachi ; Workshop of the Sarachi',
 'Pérez, Bartolomé',
 'Maella, Mariano Salvador',
 "Brueghel el Viejo, Jan ; Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)",
 'Martí Garcés de Marcilla, José',
 'Haes, Carlos de',
 'Tobar, Alonso Miguel de',
 'Balen, Hendrick van ; Brueghel el Joven, Jan ; Brueghel the Younger, Jan',
 'Camprobín, Pedro de',
 'Rincón, Fernando del',
 'Hiepes, Tomás',
 'Bertodano, Luis de',
 'Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid',
 'Vauquer, Jacques',
 'Coosemans, Alexander',
 'Leoni, Leone',
 'J. Laurent y Cía',
 'Arellano, Juan de',
 'Galleria dei Lavori, Florencia',
 'Belvedere, Andrea ; Solimena, Francesco',
 'Anonymous ; Anónimo',
 'Malaine, Joseph-Laurent',
 'Dyck, Anthony van ; Dyck, Anton van',
 'Puebla y Tolín, Dióscoro Teófilo',
 'Maestro de la Santa Sangre',
 'David, Gérard'

In [27]:
for e in nombres_unicos:
    print(e)

Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)
Maestro de la Sisla ; Master of La Sisla
Fontana, Annibale ; Taller de los Sarachi ; Workshop of the Sarachi
Pérez, Bartolomé
Maella, Mariano Salvador
Brueghel el Viejo, Jan ; Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)
Martí Garcés de Marcilla, José
Haes, Carlos de
Tobar, Alonso Miguel de
Balen, Hendrick van ; Brueghel el Joven, Jan ; Brueghel the Younger, Jan
Camprobín, Pedro de
Rincón, Fernando del
Hiepes, Tomás
Bertodano, Luis de
Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid
Vauquer, Jacques
Coosemans, Alexander
Leoni, Leone
J. Laurent y Cía
Arellano, Juan de
Galleria dei Lavori, Florencia
Belvedere, Andrea ; Solimena, Francesco
Anonymous ; Anónimo
Malaine, Joseph-Laurent
Dyck, Anthony van ; Dyck, Anton van
Puebla y Tolín, Dióscoro Teófilo
Maestro de la Santa Sangre
David, Gérard
Titian (Tiziano Vecellio) ; Tiziano, Vecellio di Gregorio
Pulido Fernández, Ramón
Pereda y Salgado, Antonio de

In [28]:
nombres_unicos[0]

'Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)'

In [29]:
descomposicion_de_materia = nombres_unicos[0].split(";")
descomposicion_de_materia

['Nuzzi, Mario ', ' Nuzzi, Mario (Mario dei Fiori)']

In [30]:
for i in descomposicion_de_materia:
    print(i)

Nuzzi, Mario 
 Nuzzi, Mario (Mario dei Fiori)


In [31]:
#%pip install fuzzywuzzy

In [32]:
from fuzzywuzzy import fuzz

In [33]:
def choose_author(names):
    name_list = [name.strip() for name in names.split(';')]
    
    # Calcula la similitud entre los conjuntos de palabras
    similarity_scores = []
    for i in range(len(name_list)):
        for j in range(i + 1, len(name_list)):
            score = fuzz.token_set_ratio(name_list[i], name_list[j])
            similarity_scores.append((i, j, score))
    
    # Ordena la lista de similitudes de mayor a menor
    similarity_scores.sort(key=lambda x: x[2], reverse=True)
    
    # Devuelve el nombre con mayor similitud
    if similarity_scores and similarity_scores[0][2] > 80:  # Puedes ajustar el umbral según tus necesidades
        return name_list[similarity_scores[0][0]]
    else:
        return names

In [34]:
obras_completo['Autor'] = obras_completo['Autor'].apply(choose_author)

In [35]:
terminos_a_eliminar = [
    'Workshop of the Sarachi',
    "Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)",
    'Brueghel the Younger, Jan',
    'Anonymous',
    'Workshop of the Miseroni',
    'Bosch, Hieronymus',
    'Master of the Virgin of the Catholic Monarchs',
    'Veronés, Paolo'
]

In [36]:
for termino in terminos_a_eliminar:
    obras_completo['Autor'] = obras_completo['Autor'].apply(lambda x: x.replace(termino, '') if termino in x else x)

# Eliminar espacios duplicados resultantes de las sustituciones
obras_completo['Autor'] = obras_completo['Autor'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Si deseas eliminar las filas donde la columna 'Autor' está vacía después de la sustitución
obras_completo = obras_completo[obras_completo['Autor'] != '']

# Asegurarte de que los cambios se apliquen al DataFrame original
obras_completo.reset_index(drop=True, inplace=True)

In [37]:
# Función para eliminar los punto y coma al comienzo o al final:
def eliminar_punto_y_coma(celda):
    # Elimina el punto y coma al comienzo o al final de la cadena
    celda = celda.lstrip(';').rstrip(';')
    return celda

# Aplica la función a la columna 'Autor'
obras_completo['Autor'] = obras_completo['Autor'].apply(eliminar_punto_y_coma)

In [38]:
# Función para eliminar los espacios en blanco al comienzo o al final:
def eliminar_espacios_en_blanco(celda):
    
    celda = celda.lstrip().rstrip()
    return celda

obras_completo['Autor'] = obras_completo['Autor'].apply(eliminar_espacios_en_blanco)

# Análisis "Escuela"

In [39]:
obras_completo['Escuela'].unique()

array(['Italiana', 'Española', 'Flamenca', 'Francesa',
       'Hispano-cartaginesa', 'Italiana ; Veneciana',
       'Castellana ; Española', 'Francesa ; Italiana', 'Holandesa',
       'Alemana', 'Española ; Madrileña', 'Italiana ; Napolitana',
       'Arte Clásico ; Roma', 'Boloñesa ; Italiana', 'Gala', nan,
       'Púnica', 'Hispano-flamenca', 'Española ; Valenciana',
       'Florentina ; Italiana', 'Inglesa', 'Sueca',
       'Castellana ; Española ; Toledana',
       'Francesa ; Italiana ; Milanesa', 'Portuguesa'], dtype=object)

In [40]:
obras_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Obra_uri     655 non-null    object
 1   Título       655 non-null    object
 2   Autor        655 non-null    object
 3   Fecha        655 non-null    object
 4   Año          655 non-null    int64 
 5   Escuela      649 non-null    object
 6   Objetos      319 non-null    object
 7   Fauna        188 non-null    object
 8   Flora        655 non-null    object
 9   Lugar        48 non-null     object
 10  Personajes   309 non-null    object
 11  Imágenes     655 non-null    object
 12  Fecha_rango  655 non-null    object
dtypes: int64(1), object(12)
memory usage: 66.7+ KB


In [41]:
obras_completo['Escuela'] = obras_completo['Escuela'].fillna('Desconocida')

In [117]:

def modificar_escuela(escuela):
    if pd.notna(escuela):
        escuela = str(escuela)  # Convertir a cadena si no es nulo
        if 'Italiana ; Veneciana' in escuela:
            return 'Italiana'
        elif 'Castellana ; Española' in escuela:
            return 'Española'
        elif 'Española ; Madrileña' in escuela:
            return 'Española'
        elif 'Italiana ; Napolitana' in escuela:
            return 'Italiana'
        elif 'Boloñesa ; Italiana' in escuela:
            return 'Italiana'
        elif 'Española ; Valenciana' in escuela:
            return 'Española'
        elif 'Florentina ; Italiana' in escuela:
            return 'Italiana'
        elif 'Castellana ; Española ; Toledana' in escuela:
            return 'Española'
        elif 'Francesa ; Italiana ; Milanesa' in escuela:
            return 'Francesa ; Italiana'
    
    return escuela

obras_completo['Escuela'] = obras_completo['Escuela'].apply(modificar_escuela)

In [118]:
obras_completo['Escuela'].unique()

array(['Italiana', 'Española', 'Flamenca', 'Francesa',
       'Hispano-cartaginesa', 'Francesa ; Italiana', 'Holandesa',
       'Alemana', 'Arte Clásico ; Roma', 'Gala', 'Desconocida', 'Púnica',
       'Hispano-flamenca', 'Inglesa', 'Sueca', 'Portuguesa'], dtype=object)

In [44]:
obras_completo.head()

,Obra_uri,Título,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,Imágenes,Fecha_rango
0,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Floreros""","Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,http://museodelprado.es/items/E22_Man-Made_Obj...,"""La Anunciación""",Maestro de la Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,https://content3.cdnprado.net/imagenes/Documen...,1495
2,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Copa de la Vendimia""","Fontana, Annibale ; Taller de los Sarachi",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Florero""","Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) ; Anémon...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1671
4,http://museodelprado.es/items/E22_Man-Made_Obj...,"""La Inmaculada Concepción""","Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,NaN,Azucena (Lilium candidum),NaN,Virgen María ; Ángel / ángeles,https://content3.cdnprado.net/imagenes/Documen...,1776


# Crear ID Obras

In [45]:
obras_completo['TituloID'] = range(len(obras_completo))

In [46]:
obras_completo.head()

,Obra_uri,Título,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,Imágenes,Fecha_rango,TituloID
0,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Floreros""","Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642,0
1,http://museodelprado.es/items/E22_Man-Made_Obj...,"""La Anunciación""",Maestro de la Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,https://content3.cdnprado.net/imagenes/Documen...,1495,1
2,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Copa de la Vendimia""","Fontana, Annibale ; Taller de los Sarachi",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600,2
3,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Florero""","Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) ; Anémon...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1671,3
4,http://museodelprado.es/items/E22_Man-Made_Obj...,"""La Inmaculada Concepción""","Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,NaN,Azucena (Lilium candidum),NaN,Virgen María ; Ángel / ángeles,https://content3.cdnprado.net/imagenes/Documen...,1776,4


In [47]:
nuevo_orden = ["TituloID", "Título", "Autor", "Fecha", "Año", "Escuela", "Objetos", "Fauna", "Flora", "Lugar", "Personajes", "Obra_uri", "Imágenes", "Fecha_rango"]
obras_completo = obras_completo.reindex(columns=nuevo_orden)

# Separar los df para exportar a SQL

In [48]:
# Crear DataFrames separados
obras = obras_completo[["TituloID", "Título", "Autor", "Fecha", "Año", "Escuela", "Obra_uri", "Imágenes", "Fecha_rango"]]
personajes = obras_completo[['Personajes', 'TituloID']]
objetos = obras_completo[['Objetos', 'TituloID']]
fauna = obras_completo[['Fauna', 'TituloID']]
flora = obras_completo[['Flora', 'TituloID']]
lugar = obras_completo[['Lugar', 'TituloID']]

In [49]:
obras.head()

,TituloID,Título,Autor,Fecha,Año,Escuela,Obra_uri,Imágenes,Fecha_rango
0,0,"""Floreros""","Nuzzi, Mario",1640 - 1642,1640,Italiana,http://museodelprado.es/items/E22_Man-Made_Obj...,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,1,"""La Anunciación""",Maestro de la Sisla,Hacia 1500,1495,Española,http://museodelprado.es/items/E22_Man-Made_Obj...,https://content3.cdnprado.net/imagenes/Documen...,1495
2,2,"""Copa de la Vendimia""","Fontana, Annibale ; Taller de los Sarachi",1570 - 1600,1570,Italiana,http://museodelprado.es/items/E22_Man-Made_Obj...,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,3,"""Florero""","Pérez, Bartolomé",Hacia 1676,1671,Española,http://museodelprado.es/items/E22_Man-Made_Obj...,https://content3.cdnprado.net/imagenes/Documen...,1671
4,4,"""La Inmaculada Concepción""","Maella, Mariano Salvador",Hacia 1781,1776,Española,http://museodelprado.es/items/E22_Man-Made_Obj...,https://content3.cdnprado.net/imagenes/Documen...,1776


In [50]:
obras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   TituloID     655 non-null    int64 
 1   Título       655 non-null    object
 2   Autor        655 non-null    object
 3   Fecha        655 non-null    object
 4   Año          655 non-null    int64 
 5   Escuela      655 non-null    object
 6   Obra_uri     655 non-null    object
 7   Imágenes     655 non-null    object
 8   Fecha_rango  655 non-null    object
dtypes: int64(2), object(7)
memory usage: 46.2+ KB


In [51]:
personajes = personajes.dropna(subset=['Personajes']).reset_index(drop=True)

In [52]:
personajes.head()

,Personajes,TituloID
0,Virgen María,1
1,Virgen María ; Ángel / ángeles,4
2,San Miguel ; Virgen María ; Ángel / ángeles,9
3,Cibeles,10
4,Jesús ; Virgen María,12


In [53]:
len(personajes)

309

In [54]:
objetos = objetos.dropna(subset=['Objetos']).reset_index(drop=True)

In [55]:
objetos.head()

,Objetos,TituloID
0,Enseres domésticos ; Florero ; Instrumentos de...,0
1,Libro,1
2,Elementos heráldicos y nobiliarios ; Insignia ...,4
3,Enseres domésticos ; Florero ; Libro ; Lámpara...,6
4,Armas ; Espada,9


In [56]:
len(objetos)

319

In [57]:
fauna = fauna.dropna(subset=['Fauna']).reset_index(drop=True)

In [58]:
fauna.head()

,Fauna,TituloID
0,Artrópodos (Filo: Arthropoda),3
1,Lobo (Canis lupus) ; Oveja (Ovis aries),9
2,Ciervo (Cervus elaphus) ; Cobaya común / Conej...,10
3,Artrópodos (Filo: Arthropoda) ; Gran pavón noc...,11
4,Paloma (Columba ssp),15


In [59]:
len(fauna)

188

In [60]:
flora = flora.dropna(subset=['Flora']).reset_index(drop=True)

In [61]:
flora.head()

,Flora,TituloID
0,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,0
1,Azucena (Lilium candidum),1
2,Vid / Parra (Vitis vinifera),2
3,Ajenuz / Arañuela (Nigella damascena) ; Anémon...,3
4,Azucena (Lilium candidum),4


In [62]:
len(flora)

655

In [63]:
lugar = lugar.dropna(subset=['Lugar']).reset_index(drop=True)

In [64]:
lugar.head()

,Lugar,TituloID
0,Alicante ; Elche ; España,8
1,España ; Las Navas ; Ávila,45
2,El Escorial ; España ; Madrid,69
3,España ; Valencia,70
4,Italia,71


In [65]:
len(lugar)

48

In [66]:
lugar['Lugar'].unique()

array(['Alicante ; Elche ; España', 'España ; Las Navas ; Ávila',
       'El Escorial ; España ; Madrid', 'España ; Valencia', 'Italia',
       'España ; Guadarrama ; Madrid', 'Francia ; Normandía',
       'Cantabria ; España ; San Vicente de la Barquera',
       'España ; Sevilla', 'Alsasua ; España ; Navarra',
       'España ; Granada', 'Asturias ; España ; Picos de Europa',
       'Italia ; Palermo ; Sicilia', 'Magreb ; Marruecos',
       'Buñol ; España ; Valencia', 'Bruselas ; Bélgica',
       'Brabante, Tervuren ; Bélgica', 'Argelia',
       'España ; Gran Canaria ; Islas Canarias',
       'Aranjuez ; España ; Madrid', 'El Pardo ; España ; Madrid',
       'Asturias ; España', 'España ; Madrid', 'Italia ; Roma',
       'Frascati ; Italia ; Roma', 'España ; Salamanca',
       'Italia ; Venecia'], dtype=object)

# Análisis de los DataFrame obtenidos.

# Personajes

In [67]:
personajes.head()

,Personajes,TituloID
0,Virgen María,1
1,Virgen María ; Ángel / ángeles,4
2,San Miguel ; Virgen María ; Ángel / ángeles,9
3,Cibeles,10
4,Jesús ; Virgen María,12


In [68]:
#personajes['Personajes'].unique()

In [69]:
#nombres_personajes = personajes['Personajes']
#nombres_personajes = list(nombres_personajes)
#nombres_personajes

In [70]:
#len(nombres_personajes)

In [71]:
# Convertir todos los elementos a cadenas antes de aplicar split
#nombres_personajes = [str(nombre) for nombre in nombres_personajes]

# Aplicar la descomposición solo a elementos de tipo string
#descomposicion_nombres_personajes = [nombre.split(";") for nombre in nombres_personajes]


#for descomposicion in descomposicion_nombres_personajes:
    #print(descomposicion)

In [72]:
#len(descomposicion_nombres_personajes)

In [73]:
#personajes['Personajes'] = descomposicion_nombres_personajes

In [74]:
personajes['Personajes'] = personajes['Personajes'].str.split(';')


In [75]:

personajes = personajes.explode('Personajes')
personajes = personajes.reset_index(drop=True)


In [76]:
personajes['Personajes'] = personajes['Personajes'].str.strip()

In [77]:
personajes['PersonajesID'] = pd.factorize(personajes['Personajes'])[0]

In [78]:
nuevo_orden = ["PersonajesID", "Personajes", "TituloID"]
personajes = personajes.reindex(columns=nuevo_orden)

In [79]:
personajes.head()

,PersonajesID,Personajes,TituloID
0,0,Virgen María,1
1,0,Virgen María,4
2,1,Ángel / ángeles,4
3,2,San Miguel,9
4,0,Virgen María,9


# Objetos

In [80]:
objetos.head()

,Objetos,TituloID
0,Enseres domésticos ; Florero ; Instrumentos de...,0
1,Libro,1
2,Elementos heráldicos y nobiliarios ; Insignia ...,4
3,Enseres domésticos ; Florero ; Libro ; Lámpara...,6
4,Armas ; Espada,9


In [81]:
objetos['Objetos'] = objetos['Objetos'].str.split(';')

In [82]:
objetos = objetos.explode('Objetos')

objetos = objetos.reset_index(drop=True)

In [83]:
objetos.head()

,Objetos,TituloID
0,Enseres domésticos,0
1,Florero,0
2,Instrumentos de cuerda / cordófonos,0
3,Instrumentos musicales,0
4,Violín,0


In [84]:
objetos['Objetos'] = objetos['Objetos'].str.strip()

In [85]:
objetos['ObjetosID'] = pd.factorize(objetos['Objetos'])[0]

In [86]:
nuevo_orden = ["ObjetosID", "Objetos", "TituloID"]
objetos = objetos.reindex(columns=nuevo_orden)

In [87]:
objetos.head()

,ObjetosID,Objetos,TituloID
0,0,Enseres domésticos,0
1,1,Florero,0
2,2,Instrumentos de cuerda / cordófonos,0
3,3,Instrumentos musicales,0
4,4,Violín,0


# Fauna

In [88]:
fauna.head()

,Fauna,TituloID
0,Artrópodos (Filo: Arthropoda),3
1,Lobo (Canis lupus) ; Oveja (Ovis aries),9
2,Ciervo (Cervus elaphus) ; Cobaya común / Conej...,10
3,Artrópodos (Filo: Arthropoda) ; Gran pavón noc...,11
4,Paloma (Columba ssp),15


In [89]:
fauna['Fauna'] = fauna['Fauna'].str.split(';')

In [90]:
fauna = fauna.explode('Fauna')

fauna = fauna.reset_index(drop=True)

In [91]:
fauna['Fauna'] = fauna['Fauna'].str.strip()

In [92]:
fauna['FaunaID'] = pd.factorize(fauna['Fauna'])[0]

In [93]:
nuevo_orden = ["FaunaID", "Fauna", "TituloID"]
fauna = fauna.reindex(columns=nuevo_orden)

In [94]:
fauna.head()

,FaunaID,Fauna,TituloID
0,0,Artrópodos (Filo: Arthropoda),3
1,1,Lobo (Canis lupus),9
2,2,Oveja (Ovis aries),9
3,3,Ciervo (Cervus elaphus),10
4,4,Cobaya común / Conejillo de Indias (Cavia porc...,10


# Flora

In [95]:
flora.head()

,Flora,TituloID
0,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,0
1,Azucena (Lilium candidum),1
2,Vid / Parra (Vitis vinifera),2
3,Ajenuz / Arañuela (Nigella damascena) ; Anémon...,3
4,Azucena (Lilium candidum),4


In [96]:
flora['Flora'] = flora['Flora'].str.split(';')

In [97]:
flora = flora.explode('Flora')

flora = flora.reset_index(drop=True)

In [98]:
flora['Flora'] = flora['Flora'].str.strip()

In [99]:
flora['FloraID'] = pd.factorize(flora['Flora'])[0]

In [100]:
nuevo_orden = ["FloraID", "Flora", "TituloID"]
flora = flora.reindex(columns=nuevo_orden)

In [101]:
flora.head()

,FloraID,Flora,TituloID
0,0,Anémona (Anemone spp.),0
1,1,Gladiolo (Gladiolus spp.),0
2,2,Lirio (Iris germanica),0
3,3,Mundillo / Bola de nieve (Viburnum opulus L.),0
4,4,Peonía (Paeonia),0


# Lugar

In [102]:
lugar.head()

,Lugar,TituloID
0,Alicante ; Elche ; España,8
1,España ; Las Navas ; Ávila,45
2,El Escorial ; España ; Madrid,69
3,España ; Valencia,70
4,Italia,71


In [103]:
lugar['Lugar'] = lugar['Lugar'].str.split(';')

In [104]:
lugar = lugar.explode('Lugar')

lugar = lugar.reset_index(drop=True)

In [105]:
lugar['Lugar'] = lugar['Lugar'].str.strip()

In [106]:
lugar['LugarID'] = pd.factorize(lugar['Lugar'])[0]

In [107]:
nuevo_orden = ["LugarID", "Lugar", "TituloID"]
lugar = lugar.reindex(columns=nuevo_orden)

In [108]:
lugar.head()

,LugarID,Lugar,TituloID
0,0,Alicante,8
1,1,Elche,8
2,2,España,8
3,2,España,45
4,3,Las Navas,45


# Eliminar columnas que no vaya a necesitar en el estudio

In [ ]:
cols_eliminar = ["TituloID", "Fecha", "Obra_uri", "Imágenes"]

# Creación de tablas intermedias

In [109]:
intermedia_titulo_objetos = objetos[['ObjetosID', 'TituloID']]
intermedia_titulo_personajes = personajes[['PersonajesID', 'TituloID']]
intermedia_titulo_fauna = fauna[['FaunaID', 'TituloID']]
intermedia_titulo_flora = flora[['FloraID', 'TituloID']]
intermedia_titulo_lugar = lugar[['LugarID', 'TituloID']]

In [110]:
intermedia_titulo_objetos.head(2)

,ObjetosID,TituloID
0,0,0
1,1,0


In [111]:
intermedia_titulo_personajes.head(2)

,PersonajesID,TituloID
0,0,1
1,0,4


In [112]:
intermedia_titulo_fauna.head(2)

,FaunaID,TituloID
0,0,3
1,1,9


In [113]:
intermedia_titulo_flora.head(2)

,FloraID,TituloID
0,0,0
1,1,0


In [114]:
intermedia_titulo_lugar.head(2)

,LugarID,TituloID
0,0,8
1,1,8


In [ ]:
lista_completa = []

# Agregar cada sublista a la lista principal
for sublist in datos:
    lista_completa.append(sublist)

# Mostrar el resultado
print(lista_completa)

In [ ]:
lista_personajes = []

# Agregar elementos de cada lista a la lista principal
for sublist in descomposicion_nombres_unicos_personajes:
    lista_personajes.extend(sublist)

In [ ]:
# Mostrar el resultado
lista_personajes

In [ ]:
def limpiar_espacios(cadena):
    return cadena.strip()


# Eliminar espacios al principio y al final de cada cadena en cada sublista
lista_sin_espacios = [[limpiar_espacios(elemento) for elemento in sublista] for sublista in lista_personajes]

# Mostrar el resultado
for sublista in lista_personajes:
    print(sublista)

# Exportación de los DataFrames

In [115]:
obras_completo.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/obras_completo.csv", index=False)

In [116]:
obras.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/obras.csv", index=False)
#personajes.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/personajes.csv", index=False)
#objetos.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/objetos.csv", index=False)
#fauna.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/fauna.csv", index=False)
#flora.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/flora.csv", index=False)
#lugar.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/lugar.csv", index=False)

In [ ]:
intermedia_titulo_objetos.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/intermedia_titulo_objetos.csv", index=False)
intermedia_titulo_personajes.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/intermedia_titulo_personajes.csv", index=False)
intermedia_titulo_fauna.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/intermedia_titulo_fauna.csv", index=False)
intermedia_titulo_flora.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/intermedia_titulo_flora.csv", index=False)
intermedia_titulo_lugar.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_clean/intermedia_titulo_lugar.csv", index=False)